# Mesh based surrogate modeling

## GNN

GNNs leverage the structure and features of nodes and edges to achieve one of **three* tasks:

1. node classification: predicting unknown quantities for the nodes of the graph
1. link prediction: predicting the existence of missing links between nodes
1. graph classification: predicting unknowns for the entire graph

Whichever the task, the key idea in GNNs is to learn how to propagate information into nodes from their local neighborhoods across interconnecting edges for all nodes in the graph.


 The properties of GNNs — structural equivariance, scaling, high dependence on local structure and neighbor embeddings — are all intuitively linked to the irregularly structured, neighboring region-dependent dynamics of flow systems.


 ## Model design

* **Node type**: a 9-dimensional one-hot vector corresponding to node location in fluid, wall, inflow, or outflow regions.
* **Mesh topology**: each node contains the 2D position vector of its location in the two dimensional space that is being simulated.
* **Node attributes**: this is the underlying data that we will be training MeshGraphNets to simulate. The dataset contains both the 2D velocity vector of the fluid (as illustrated in Fig.6), and the scalar pressure, which can be directly calculated from the time-dependent velocity.

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*vHTXvpAxfwdNvA-HPvFvCQ.png' />
datset: https://drive.google.com/drive/folders/1QANENxeWRVBs2TZ8SQ5CGuHo27i95WtO?usp=sharing

in PyG, each graph is stored in an instance of torch_geometric.data.Data, which stores all of the attributes of the graph. Our pre-processor returns a list of all the graphs at each time step for all trajectories used during training. A single element (i.e., a graph corresponding to a single time step), has the following signature:

```python
'''
For each node i in the graph with neighbor node j:
x - (the node features) contains the ground truth 2D velocities concatenated with the 9D node type one-hot vector for all nodes [num_nodes x 11]
edge_index - contains the connectivity of the graph in COO format. [2 x num_edges]
edge_attr - contains the 2D position vector, u_ij, between connecting nodes (u_ij=u_i-u_j) concatenated with the 2-norm of the position vector. [num_edges x 3]
y - (the node outputs) contains the fluid acceleration between the current graph and the graph in the next time step. These are the features used for training: y=(v_t_next-v_t_curr)/dt [num_nodes x 2]
p - pressure scalar, used for validation [num_nodes x 1]
cells and mesh_pos: these attributes contain no new information. They are included for ease of visualization of the results and explained in detail in the Colab.
'''
Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, p=p,
                                  cells=cells, mesh_pos=mesh_pos)

```

We demonstrate its use as a dynamic forward fluid flow simulation surrogate model for a meshed domain: given the velocities at each node at a time step, it predicts the acceleration at each mesh node, from which we can calculate the next step’s velocities through forward integration. The architecture of MeshGraphNets consists of encoding, processing, and decoding steps. “Encoding” generates node and edge embeddings from features, “processing” takes care of message passing, aggregation, and updating, and decoding is the post-processing step that gives final predictions
<img src='./MeshGrapNet-overview.webp'>

### Encoding

### Processing

### Decoding


## loss function

We only calculate loss for and optimize over normal and outflow node types in the mesh: we don’t calculate flow acceleration error for walls

## Resources

* [Learning Mesh-Based Simulation with Graph Networks] https://ar5iv.labs.arxiv.org/html/2010.03409?_immersive_translate_auto_translate=0 
* [Learning Reduced-Order Models for Cardiovascular Simulations with Graph Neural Networks ]https://arxiv.org/pdf/2303.07310.pdf : Their analysis, however, is limited to time-independent PDEs and does not transfer to dynamical systems.
* https://arxiv.org/pdf/2308.01602.pdf [paper]
* https://medium.com/stanford-cs224w/learning-mesh-based-flow-simulations-on-graph-networks-44983679cf2d [demo]

Our idea for the present work is to exploit the capabilities of GNNs in order to learn a nonintrusive data-driven time-stepping scheme for evolving high dimensional parameter dependent dynamical systems.